In [1]:
from tika import parser
import re
import pprint
#Question Asked	Answer Given	Top Match	Answer Taught	Category	Question Type	Date Received	 Date Responded	Response Time	Processed By

pp = pprint.PrettyPrinter(indent=2)

def get_content(fp):
    raw = parser.from_file(fp)

    stopwords = set(['\n', '\xa0', '\ue010', '\ue011'])
    content = raw['content'].strip()
    content = content.split('\n')
    content = [c.replace(u'\xa0', u' ') for c in content]
    content = [c for c in content if c != '']
    content = [x for x in content if x not in stopwords]
    
    return content
    
def get_question(content):
    found = False
    question = []
    for i in range(len(content)):
        if not found:
            search_groups = re.search('^Q (.*)', content[i])
            if search_groups is not None:
                found = True
                qn = search_groups.group(1).strip()
                question.append(qn)
        else:
            if re.match('^A(?!.)|^A (.*)', content[i]):
                return ' '.join(question), i
                continue
            else:
                question.append(content[i])
    return None, 0

def get_answer(content):
    found = False
    answer = []
    for i in range(len(content)):
        if not found:
            search_groups = re.search('^A(.*)', content[i])
            if search_groups is not None:
                found = True
                ans = search_groups.group(1).strip()
                answer.append(ans)
        else:
            if re.match('^Answer', content[i]):
                return ' '.join(answer), i
                continue
            else:
                answer.append(content[i])
    return None, 0

def get_answer_name(content):
    found = False
    answer = []
    for i in range(len(content)):
        if not found:
            search_groups = re.search('^Answer Name (.*)', content[i])
            if search_groups is not None:
                found = True
                ans = search_groups.group(1).strip()
                answer.append(ans)
        else:
            if re.match('^Contextual', content[i]):
                return ' '.join(answer), i
                continue
            else:
                answer.append(content[i])
    return None, 0

_fields = { 
          'answer': 'DEFAULT_ANSWER',
          'answer_name': 'DEFAULT_ANSWER_NAME',
          'category': 'DEFAULT_CATEGORY',
          'created_by': 'DEFAULT_CREATED_BY',
          'created_date': 'DEFAULT_CREATED_DATE',
          'edited_by': 'DEFAULT_EDITED_BY',
          'edited_date': 'DEFAULT_EDITED_DATE',
          'question': 'DEFAULT_QUESTION'
          }

def get_meta(fp):
    content = get_content(fp)
    _fields['question'], i = get_question(content)
    _fields['answer'], i = get_answer(content[i:])
    _fields['answer_name'], i = get_answer_name(content[i:])
    return _fields

def get_phrases(fp):
    content = get_content(fp)
    # Retrieving training phrases
#     indices = [i for i, item in enumerate(content) if re.match('^([0-9]+)(.*)([0-9]+)$', item)]
#     if len(indices) > 0:
#         start_index = indices[0]
#         end_index = indices[-1]
#     else:
#         return []

    indices = [i for i, item in enumerate(content) if re.match('(Question Text Count Reteach)|(flexAnswer Professional v8.8)', item)]
    try:
        start_index = indices[0] + 1
        end_index= indices[1]
    except Exception:
        return []
#     try:
#         start_index = content.index('Question Text Count Reteach') + 1
#         end_index = content.index('flexAnswer Professional v8.8 ')
#     except Exception:
#         start_index = content.index('Question Text Count Reteach') + 1
#         end_index = content.index('flexAnswer Professional v8.8')
    phrases = content[start_index:end_index]
    new_phrases = []
    for phrase in phrases:
        group_1 = re.search('^[0-9]+(.*)[0-9]+$', phrase)
        group_2 = re.search('^[0-9]+(.*)[0-9]*$', phrase)
        if group_1 is not None:
            phr = group_1.group(1).strip()
            new_phrases.append(phr)
        elif group_2 is not None:
            phr = group_2.group(1).strip()
            new_phrases.append(phr)
        else:
            if len(new_phrases) > 0:
                new_phrases[-1] += phrase
            else:
                new_phrases.append(phrase)
    new_phrases = [c for c in new_phrases if c != '']
    return new_phrases

## Start here if you're implementing your own scraper

In [2]:
get_content(fp)

NameError: name 'fp' is not defined

In [ ]:
import pandas as pd

# For removing duplicatess
df = pd.read_csv('datasets/askjaime3months.csv')
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
qn_asked =  set(df.question_asked)
qn_asked

In [ ]:
import os
import natsort

import pandas as pd

pdf_folders = natsort.natsorted(os.listdir('pdfs'))
pdf_files = dict()
for folder in pdf_folders:
    files = os.listdir('pdfs'+os.path.sep+folder)
    files = natsort.natsorted(files)
    pdf_files[folder] = files

phrases_df = pd.DataFrame()
phrases_count = dict()
intents = set()
intents_df = pd.DataFrame()

PHRASE_CHAR_LIMIT = 768
PHRASE_COUNT_LIMIT = 2000

for folder in pdf_files.keys():
    for file in pdf_files[folder]:
        fp = 'pdfs'+os.path.sep+folder+os.path.sep+file
        search_groups = re.search('(Q[0-9]*)(-([0-9]*))*\.pdf', file)
        if search_groups is not None:
            q_no = search_groups.group(1)
        if q_no not in phrases_count:
            phrases_count[q_no] = 0
        if q_no not in intents:
            intents.add(q_no)
            meta = get_meta(fp)
#             intents_df = intents_df.append({'intent': q_no, 'response': meta['answer']}, ignore_index=True)
            intents_df = intents_df.append({'intent': q_no, 'response': meta['answer_name']}, ignore_index=True)
        phrases = get_phrases(fp)
        for phrase in phrases:
            if phrases_count[q_no] < PHRASE_COUNT_LIMIT:
                if phrase not in qn_asked:
                    phrases_df = phrases_df.append({'intent': q_no, 'phrase': phrase[:PHRASE_CHAR_LIMIT]}, ignore_index=True)
                    phrases_count[q_no] += 1
            else:
                break

print('Intents:', len(intents_df), 'Phrases:', len(phrases_df))
intents_df.drop_duplicates(keep=False,inplace=True) 
phrases_df.drop_duplicates(keep=False,inplace=True)

print('Intents:', len(intents_df), 'Phrases:', len(phrases_df))
intents_df.to_csv('intents.csv', index=False)
phrases_df.to_csv('phrases.csv', index=False)